In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from qiskit_metal import draw, Dict,designs
from qiskit_metal.qlibrary.core import BaseQubit
from qiskit_metal.toolbox_metal import math_and_overrides
from qiskit_metal.qlibrary.core import QComponent
from qiskit_metal.draw import LineString
from qiskit_metal import MetalGUI, Dict, Headings
from qiskit_metal.qlibrary.core.qroute import QRouteLead, QRoutePoint, QRoute
from qiskit_metal.qlibrary.qubits.transmon_cross import TransmonCross
from qiskit_metal.qlibrary.resonator.readoutres_fc import ReadoutResFC
from  qiskit_metal.qlibrary.user_components.my_qcomponent import  New_Transomon_Cross, RouteConnector,MyReadoutRes01,MyFluxLine01,MyFluxLine02,MyConnector,MyXYLine01
from  qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.tlines.pathfinder import RoutePathfinder
from qiskit_metal.qlibrary.terminations.launchpad_wb import LaunchpadWirebond
import  math
from collections import  OrderedDict
import numpy as np

In [3]:
# Initialise design
design = designs.DesignFlipChip()
# Specify design name
design.metadata['design_name'] = 'FlipChip_Device'
# launch GUI
gui = MetalGUI(design)
# Allow running the same cell here multiple times to overwrite changes
design.overwrite_enabled = True

In [4]:
# design.chips.Q_chip.size_x = '12.5 mm'
# design.chips.Q_chip.size_y = '12.5 mm'
# design.chips.C_chip.size_x = '12.5 mm'
# design.chips.C_chip.size_y = '12.5 mm'
design.chips['Q_chip']['material'] = 'sapphire'
design.chips['C_chip']['material'] = 'sapphire'
design.chips['Q_chip']['size']['size_x'] = '8 mm'
design.chips['Q_chip']['size']['size_y'] = '8 mm'
design.chips['C_chip']['size']['size_x'] = '15 mm'
design.chips['C_chip']['size']['size_y'] = '15 mm'
# float(design.chips['Q_chip']['size']['size_x'][0])
design.variables.cpw_gap='5 um'
design.variables

{'cpw_width': '10 um',
 'cpw_gap': '5 um',
 'sample_holder_top': '890um',
 'sample_holder_bottom': '1650um'}

In [5]:
# design the layout of launchpad
points = []
N=14
size = 10.0
for i in range(N):
    shape = draw.Point(-size/2+i*size/N,size/2)
    points.append(shape)
# for i in range(N):
#     shape = draw.Point(size/2,size/2-i*size/N)
x = draw.shapely.geometrycollections(points)
x0 = draw.rotate(x,90,origin=(0,0))
x1 = draw.rotate(x0,90,origin=(0,0))
x2 = draw.rotate(x1,90,origin=(0,0))
square = draw.shapely.geometrycollections([x,x0,x1,x2])
square = draw.rotate(square,45,origin=(0,0))
square_coords =[]
for i in range(4):
    for j in range(N):
        square_coords.append(square.geoms[i].geoms[j].coords[0])
# square_coords[15]
# square.geoms[1]

In [6]:
opt=Dict(pos_x=0 , pos_y=0, orientation='-45', pad_width='245 um', pad_height='245 um', pad_gap = '100 um', lead_length = '176 um', chip = 'C_chip')
opt_a=Dict(pos_x=0 , pos_y=0, orientation='45', pad_width='245 um', pad_height='245 um', pad_gap = '100 um', lead_length = '176 um', chip = 'C_chip')
opt_b=Dict(pos_x=0 , pos_y=0, orientation='135', pad_width='245 um', pad_height='245 um', pad_gap = '100 um', lead_length = '176 um', chip = 'C_chip')
opt_c=Dict(pos_x=0 , pos_y=0, orientation='-135', pad_width='245 um', pad_height='245 um', pad_gap = '100 um', lead_length = '176 um', chip = 'C_chip')
# test = OpenToGround(design, 'open01', options=Dict(pos_x='-3 mm',  pos_y=pos_y_zline+0.02, orientation='-45', chip ='C_chip'),)
launch_zline = LaunchpadWirebond(design,'launch_zline',options=opt)
launch_zline_a = LaunchpadWirebond(design,'launch_zline_a',options=opt_a)
launch_zline_b = LaunchpadWirebond(design,'launch_zline_b',options=opt_b)
launch_zline_c = LaunchpadWirebond(design,'launch_zline_c',options=opt_c)
gui.rebuild()
gui.autoscale()

In [7]:
launch_zline.options

{'pos_x': 0,
 'pos_y': 0,
 'orientation': '-45',
 'chip': 'C_chip',
 'layer': '1',
 'trace_width': 'cpw_width',
 'trace_gap': 'cpw_gap',
 'lead_length': '176 um',
 'pad_width': '245 um',
 'pad_height': '245 um',
 'pad_gap': '100 um',
 'taper_height': '122um'}

In [8]:
design.delete_all_components()
launch_list = []
for i in range(4):
    for j in range(N):
         if (i==0):
            launch_list.append(design.copy_qcomponent(launch_zline,'launch_zline'+str(i)+str(j), Dict(pos_x =square_coords[i*N+j][0] ,pos_y=square_coords[i*N+j][1])))
         elif (i==1):
             launch_list.append(design.copy_qcomponent(launch_zline_a,'launch_zline'+str(i)+str(j), Dict(pos_x =square_coords[i*N+j][0] ,pos_y=square_coords[i*N+j][1])))
         elif (i==2):
             launch_list.append(design.copy_qcomponent(launch_zline_b,'launch_zline'+str(i)+str(j), Dict(pos_x =square_coords[i*N+j][0] ,pos_y=square_coords[i*N+j][1])))
         else:
             launch_list.append(design.copy_qcomponent(launch_zline_c,'launch_zline'+str(i)+str(j), Dict(pos_x =square_coords[i*N+j][0] ,pos_y=square_coords[i*N+j][1])))
# launch_list.append(design.copy_qcomponent(launch_zline,'launch_zline'+str(15), Dict(pos_x =square_coords[15][0] ,pos_y=square_coords[15][1])))
gui.rebuild()
gui.autoscale()

In [9]:
# # delete existing components
# design.delete_all_components()

# draw an Xmon on the Q_chip. Notice that I have defined one more item called chip and set it to be the 'Q_chip'
# position the center of the Xmon at (0,0)
q0_x = '0.0 mm'
q0_y = '0.0 mm'

# build the device, positioned at (q1_x, q1_y)
# q1 = TransmonCross(design, 'Q1', options = Dict(pos_x=q1_x, pos_y=q1_y, **options))
q0 = New_Transomon_Cross(design, 'Q0', options = Dict(pos_x=q0_x, pos_y=q0_y, chip='Q_chip'))

# rebuild in gui
gui.rebuild()
gui.autoscale()

In [10]:
q0.options.gds_cell_name = 'FakeJunction_01'


In [11]:
#design the n qubits layout (n must be even)
qq_space = 0.015
qubit_num = 10
qubit_pos_list = []
y1 = q0.parse_options().cross_width+qq_space
y2 = -1*(q0.parse_options().cross_width+qq_space)
x1 = -int(qubit_num/4)*y1
for i in range(int(qubit_num)):
    if  (i<qubit_num/2):
        qubit_pos_list.append((x1+i*y1,y1))
    else:
        qubit_pos_list.append((x1+(i-qubit_num/2)*y1,y2))
qubit_pos_list

[(-2.71, 1.355),
 (-1.355, 1.355),
 (0.0, 1.355),
 (1.3549999999999995, 1.355),
 (2.71, 1.355),
 (-2.71, -1.355),
 (-1.355, -1.355),
 (0.0, -1.355),
 (1.3549999999999995, -1.355),
 (2.71, -1.355)]

In [12]:
qubit_list = []
for i in range(int(qubit_num)):
     qubit_list.append(design.copy_qcomponent(q0,'Q'+str(i+1), Dict(pos_x =qubit_pos_list[i][0] ,pos_y=qubit_pos_list[i][1])))
gui.rebuild()
gui.autoscale()

In [13]:
# draw a readout resonator on the C_chip. Note that we specify the chip='C_chip'.

options = Dict(
       readout_radius='50 um',
       readout_cpw_width='10 um',
       readout_cpw_gap='5 um',
       readout_cpw_turnradius='27 um',
       vertical_start_length = '40 um',
       vertical_end_length = '300 um',
       horizontal_start_length01= '400 um',
       horizontal_start_length02 = '400 um',
       horizontal_end_length = '500 um',
       total_length = '4200 um',
       arc_step='1 um',
       meander_round = '5',
       orientation='0',
       layer='1',
       layer_subtract='2',
       inverse = False,
       mirror = False,
       subtract=True,
       chip='C_chip ',)
location_x = qubit_list[0].parse_options().cross_width/4
# the resonator is set to have its origin at the center of the circular patch.
# So we set the qubit and the resonator to share the same coordinate (q1_x, q1_y)
r1 = MyReadoutRes01(design, 'R1', options = Dict(pos_x = qubit_pos_list[0][0]+location_x, pos_y = qubit_pos_list[0][1], **options))
# rebuild in gui
gui.rebuild()
gui.autoscale()

In [14]:
# draw first five resonators
resonator_list = []
resonator_list.append(r1)
for i in range(int(qubit_num/2)-1):
     location = qubit_list[i+1].parse_options().cross_width/4
     resonator_list.append(design.copy_qcomponent(r1,'R'+str(i+2), Dict(pos_x =qubit_pos_list[i+1][0]+location,pos_y=qubit_pos_list[i+1][1])))
gui.rebuild()
gui.autoscale()

In [15]:
# draw next five resonators
options = Dict(
       readout_radius='50 um',
       readout_cpw_width='10 um',
       readout_cpw_gap='5 um',
       readout_cpw_turnradius='27 um',
       vertical_start_length = '40 um',
       vertical_end_length = '300 um',
       horizontal_start_length01= '400 um',
       horizontal_start_length02 = '400 um',
       horizontal_end_length = '500 um',
       total_length = '4200 um',
       arc_step='1 um',
       meander_round = '5',
       orientation='0',
       layer='1',
       layer_subtract='2',
       inverse = True,
       mirror = False,
       subtract=True,
       chip='C_chip ',)
location_x = qubit_list[5].parse_options().cross_width/4
# the resonator is set to have its origin at the center of the circular patch.
# So we set the qubit and the resonator to share the same coordinate (q1_x, q1_y)
r1 = MyReadoutRes01(design, 'R6', options = Dict(pos_x = qubit_pos_list[5][0]-location_x, pos_y = qubit_pos_list[5][1], **options))
# rebuild in gui
gui.rebuild()
gui.autoscale()

In [16]:
resonator_list.append(r1)
for i in range(int(qubit_num/2)-1):
     location = qubit_list[i+1].parse_options().cross_width/4
     resonator_list.append(design.copy_qcomponent(r1,'R'+str(i+2+int(qubit_num/2)), Dict(pos_x =qubit_pos_list[i+1+int(qubit_num/2)][0]-location,pos_y=qubit_pos_list[i+1+int(qubit_num/2)][1])))
gui.rebuild()
gui.autoscale()

In [17]:
design.components['launch_zline02'].pins
design.components['R1'].pins.readout.middle[1]

0.56

In [18]:
# draw first readout line
res_readout_y = design.components['R1'].pins.readout.middle[1]
rr_space = 0.025
readout01_y = res_readout_y-rr_space
Q_chip_size =float(design.chips['Q_chip']['size']['size_x'][0])
extended_pin_length = 0.09
fillet = 0.09

def  get_extended_pos_y(y1,length,ydirction):
    y_extended = y1+length*ydirction
    return y_extended

def  get_connector_pos_xy(y,size,type):
    if (type==1):
        x = y-size*math.sqrt(2)/2
    elif (type==2):
        x = size*math.sqrt(2)/2-y
    elif (type==3):
        x = y+size*math.sqrt(2)/2
    else:
        x = -size*math.sqrt(2)/2-y

    return x,y

y_extended_launch_zline02 =get_extended_pos_y(design.components['launch_zline02'].pins.tie.middle[1],extended_pin_length,
                                              design.components['launch_zline02'].pins.tie.normal[1])

readout01_c_pos_x_l, readout01_c_pos_y_l = get_connector_pos_xy(y_extended_launch_zline02,Q_chip_size,1)

opt = Dict(pos_x=readout01_c_pos_x_l,
                           pos_y=readout01_c_pos_y_l,
                           width='10 um',
                           gap='7 um',
                           width0='10 um',
                           gap0='5 um',
                           length = '100 um',
                           orientation='0',
                           layer='1',
                           layer_subtract='2',
                           subtract=True,
                           chip='C_Chip', )
my_connector_readout01_l = MyConnector(design,'my_connector_readout01_l', options=opt)

pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='my_connector_readout01_l',pin='c_pin_l'),
                         end_pin=Dict(component='launch_zline02',pin='tie'),),lead = Dict(end_straight = extended_pin_length,),
                          chip = 'C_chip')
readout01_lz02=RoutePathfinder(design,'readout01_lz02',options=pin_opt)

y_extended_launch_zline312 =get_extended_pos_y(design.components['launch_zline312'].pins.tie.middle[1],extended_pin_length,
                                              design.components['launch_zline312'].pins.tie.normal[1])
readout01_c_pos_x_r, readout01_c_pos_y_r = get_connector_pos_xy(y_extended_launch_zline312,Q_chip_size,2)

opt = Dict(pos_x=readout01_c_pos_x_r,
                           pos_y=readout01_c_pos_y_r,
                           width='10 um',
                           gap='5 um',
                           width0='10 um',
                           gap0='7 um',
                           length = '100 um',
                           orientation='0',
                           layer='1',
                           layer_subtract='2',
                           subtract=True,
                           chip='C_Chip', )
my_connector_readout01_r = MyConnector(design,'my_connector_readout01_r', options=opt)

pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='my_connector_readout01_r',pin='c_pin_r'),
                         end_pin=Dict(component='launch_zline312',pin='tie'),),lead = Dict(end_straight = extended_pin_length,),
                          fillet=fillet, chip = 'C_chip')
readout01_lz312=RoutePathfinder(design,'readout01_lz312',options=pin_opt)


anchors_readout01_pos_y = design.components['R1'].pins.readout.middle[1]-rr_space
anchors_readout01_pos_x01 = readout01_c_pos_x_l+0.4
anchors_readout01_pos_x02 = readout01_c_pos_x_r-0.4
anchors = OrderedDict()
anchors[0] = np.array([anchors_readout01_pos_x01,anchors_readout01_pos_y])
anchors[1]=np.array([anchors_readout01_pos_x02,anchors_readout01_pos_y])
lead = Dict(start_straight=0.2,end_straight=0.2,)
options = Dict( pin_inputs=Dict(start_pin=Dict(component='my_connector_readout01_l',pin='c_pin_r'),
                         end_pin=Dict(component='my_connector_readout01_r',pin='c_pin_l'),),anchors=anchors,
                          lead=lead, fillet=fillet, trace_gap='7um', chip = 'C_chip')
lc_readout01_rc = RoutePathfinder(design,'lc_readout01_rc',options=options)


# pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='launch_zline01',pin='tie'),
#                          end_pin=Dict(component='launch_zline313',pin='tie'),),
#                           trace_width='10um',trace_gap='6um',chip = 'C_chip')
# readout01=RoutePathfinder(design,'Readout01',options=pin_opt)

# pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='launch_zline113',pin='tie'),
#                          end_pin=Dict(component='launch_zline21',pin='tie'),),
#                           trace_width='10um',trace_gap='6um',chip = 'C_chip')
# readout02=RoutePathfinder(design,'Readout02',options=pin_opt)
gui.rebuild()
gui.autoscale()

# anchors_readout01_pos_y,anchors_readout01_pos_x01,anchors_readout01_pos_x02

In [19]:
# draw second readout line
res_readout_y = design.components['R6'].pins.readout.middle[1]
# rr_space = 0.025
# readout02_y = res_readout_y+rr_space

# extended_pin_length = 0.09
# fillet = 0.09

y_extended_launch_zline112 =get_extended_pos_y(design.components['launch_zline112'].pins.tie.middle[1],extended_pin_length,
                                              design.components['launch_zline112'].pins.tie.normal[1])

readout02_c_pos_x_l, readout02_c_pos_y_l = get_connector_pos_xy(y_extended_launch_zline112,Q_chip_size,4)

opt = Dict(pos_x=readout02_c_pos_x_l,
                           pos_y=readout02_c_pos_y_l,
                           width='10 um',
                           gap='7 um',
                           width0='10 um',
                           gap0='5 um',
                           length = '100 um',
                           orientation='0',
                           layer='1',
                           layer_subtract='2',
                           subtract=True,
                           chip='C_Chip', )
my_connector_readout02_l = MyConnector(design,'my_connector_readout02_l', options=opt)

pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='my_connector_readout02_l',pin='c_pin_l'),
                         end_pin=Dict(component='launch_zline112',pin='tie'),),lead = Dict(end_straight = extended_pin_length,),
                          chip = 'C_chip')
readout01_lz112=RoutePathfinder(design,'readout01_lz112',options=pin_opt)

y_extended_launch_zline22 =get_extended_pos_y(design.components['launch_zline22'].pins.tie.middle[1],extended_pin_length,
                                              design.components['launch_zline22'].pins.tie.normal[1])
readout02_c_pos_x_r, readout02_c_pos_y_r = get_connector_pos_xy(y_extended_launch_zline22,Q_chip_size,3)

opt = Dict(pos_x=readout02_c_pos_x_r,
                           pos_y=readout02_c_pos_y_r,
                           width='10 um',
                           gap='5 um',
                           width0='10 um',
                           gap0='7 um',
                           length = '100 um',
                           orientation='0',
                           layer='1',
                           layer_subtract='2',
                           subtract=True,
                           chip='C_Chip', )
my_connector_readout02_r = MyConnector(design,'my_connector_readout02_r', options=opt)

pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='my_connector_readout02_r',pin='c_pin_r'),
                         end_pin=Dict(component='launch_zline22',pin='tie'),),lead = Dict(end_straight = extended_pin_length,),
                          fillet=fillet, chip = 'C_chip')
readout01_lz22=RoutePathfinder(design,'readout01_lz22',options=pin_opt)

rr_space = 0.025
anchors_readout02_pos_y = design.components['R6'].pins.readout.middle[1]+rr_space
anchors_readout02_pos_x01 = readout02_c_pos_x_l+0.4
anchors_readout02_pos_x02 = readout02_c_pos_x_r-0.4
anchors = OrderedDict()
anchors[0] = np.array([anchors_readout02_pos_x01,anchors_readout02_pos_y])
anchors[1]=np.array([anchors_readout02_pos_x02,anchors_readout02_pos_y])
lead = Dict(start_straight=0.2,end_straight=0.2,)
options = Dict( pin_inputs=Dict(start_pin=Dict(component='my_connector_readout02_l',pin='c_pin_r'),
                         end_pin=Dict(component='my_connector_readout02_r',pin='c_pin_l'),),anchors=anchors,
                          lead=lead, fillet=fillet, trace_gap='7um', chip = 'C_chip')
lc_readout02_rc = RoutePathfinder(design,'lc_readout02_rc',options=options)


# pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='launch_zline01',pin='tie'),
#                          end_pin=Dict(component='launch_zline313',pin='tie'),),
#                           trace_width='10um',trace_gap='6um',chip = 'C_chip')
# readout01=RoutePathfinder(design,'Readout01',options=pin_opt)

# pin_opt = Dict( pin_inputs=Dict(start_pin=Dict(component='launch_zline113',pin='tie'),
#                          end_pin=Dict(component='launch_zline21',pin='tie'),),
#                           trace_width='10um',trace_gap='6um',chip = 'C_chip')
# readout02=RoutePathfinder(design,'Readout02',options=pin_opt)
gui.rebuild()
gui.autoscale()

In [20]:
# launch_list[4].pins
lc_readout01_rc.options

{'chip': 'C_chip',
 'layer': '1',
 'pin_inputs': {'start_pin': {'component': 'my_connector_readout01_l',
   'pin': 'c_pin_r'},
  'end_pin': {'component': 'my_connector_readout01_r', 'pin': 'c_pin_l'}},
 'fillet': 0.09,
 'lead': {'start_straight': 0.2,
  'end_straight': 0.2,
  'start_jogged_extension': '',
  'end_jogged_extension': ''},
 'total_length': '7mm',
 'trace_width': 'cpw_width',
 'anchors': {0: array([-4.43479211,  0.535     ]),
  1: array([4.43479211, 0.535     ])},
 'advanced': {'avoid_collision': 'true'},
 'step_size': '0.25mm',
 'hfss_wire_bonds': False,
 'q3d_wire_bonds': False,
 'trace_gap': '7um',
 '_actual_length': '9.989195177646161 mm'}

In [21]:
#draw z line with connector
#calculate y position for z(flux) control line connection between launch and qubit
y_extended_launch_zline04 = get_extended_pos_y(design.components['launch_zline04'].pins.tie.middle[1],extended_pin_length,design.components['launch_zline04'].pins.tie.normal[1])
# y_1 = design.components['launch_zline04'].pins.tie.middle[1]+extended_pin_length*design.components['launch_zline04'].pins.tie.normal[1]
y1_zline = y_extended_launch_zline04-qubit_pos_list[0][1]
zline01_c_pos_x, zline01_c_pos_y = get_connector_pos_xy(y_extended_launch_zline04,Q_chip_size,1)

opt = Dict(pos_x=zline01_c_pos_x,
                           pos_y=zline01_c_pos_y,
                           width='10 um',
                           gap='7 um',
                           width0='10 um',
                           gap0='5 um',
                           length = '100 um',
                           orientation='0',
                           layer='1',
                           layer_subtract='2',
                           subtract=True,
                           chip='C_Chip', )
my_connector_zline01 = MyConnector(design,'my_connector_zline01',options=opt)

opt = Dict(pos_x=qubit_pos_list[0][0], pos_y=qubit_pos_list[0][1] ,  end_yposition=y1_zline,
           inverse = True, mirror = True, flux_cpw_gap0 = '7 um', end_horizontal_length = '100 um')
my_flux01 = MyFluxLine02(design,'my_flux01',options=opt)

pin_opt = Dict(pin_inputs=Dict(start_pin=Dict(component='my_flux01',pin='flux_pin'),
                         end_pin=Dict(component='my_connector_zline01',pin='c_pin_r'),),trace_width = '10 um',
                        trace_gap='7 um',fillet=fillet, chip = 'C_chip')
zline_q1 = RouteStraight(design, 'zline_q1', options=pin_opt)

pin_opt = Dict(pin_inputs=Dict(start_pin=Dict(component='my_connector_zline01',pin='c_pin_l'),
                         end_pin=Dict(component='launch_zline04',pin='tie'),),lead = Dict(end_straight = extended_pin_length,),
                         trace_width = '10 um', trace_gap='5 um',fillet=fillet, chip = 'C_chip')
zline_lz04 = RouteStraight(design, 'zline_lz04', options=pin_opt)


# y1_zline
# opt=Dict(pos_x=-3 , pos_y=pos_y_zline, orientation='-45', pad_width='245 um', pad_height='245 um', pad_gap = '100 um', lead_length = '176 um', chip = 'C_chip')
# # test = OpenToGround(design, 'open01', options=Dict(pos_x='-3 mm',  pos_y=pos_y_zline+0.02, orientation='-45', chip ='C_chip'),)
# launch_zline = LaunchpadWirebond(design,'launch_zline',options=opt)
# end_jog_length = 0.02
# launch_zline.options.pos_y = pos_y_zline+launch_zline.parse_options().lead_length*math.cos(launch_zline.parse_options().orientation/180*math.pi)+end_jog_length*math.cos(launch_zline.parse_options().orientation/180*math.pi)
#
#
gui.rebuild()
gui.autoscale()

In [22]:
# draw xy line with connector

xyline_x0_1 = qubit_pos_list[0][0]-1/3*qubit_list[0].parse_options().cross_width
xyline_y0_1 = qubit_pos_list[0][1]
opt = Dict(pos_x=xyline_x0_1,
                       pos_y=xyline_y0_1,
                       l_1='100 um',
                       l_2='150 um',
                       l_3 = '50 um',
                       flux_cpw_width='5 um',
                       flux_cpw_gap='3 um',
                       flux_cpw_width0='10 um',
                       flux_cpw_gap0='7 um',
                       c_length = '15 um',
                       angle = '-90',
                       fillet = '1 um',
                       orientation='90',
                       layer='1',
                       layer_subtract='2',
                       inverse = False,
                       mirror = False,
                       subtract=True,
                       chip='C_Chip',
                     )

my_xyline01 = MyXYLine01(design,'my_xyline01',options=opt)

xyline_length = my_xyline01.parse_options().l_1
xyline01_c_pos_x, xyline01_c_pos_y = get_connector_pos_xy(qubit_pos_list[0][1]-xyline_length,Q_chip_size,1)
extended_pin_length_xy= (qubit_pos_list[0][1]-xyline_length-design.components['launch_zline03'].pins.tie.middle[1])/design.components['launch_zline03'].pins.tie.normal[1]
opt = Dict(pos_x=xyline01_c_pos_x,
                           pos_y=xyline01_c_pos_y,
                           width='10 um',
                           gap='7 um',
                           width0='10 um',
                           gap0='5 um',
                           length = '100 um',
                           orientation='0',
                           layer='1',
                           layer_subtract='2',
                           subtract=True,
                           chip='C_Chip', )
my_connector_xyline01 = MyConnector(design,'my_connector_xyline01',options=opt)

opts = Dict(pin_inputs=Dict(start_pin=Dict(component='my_xyline01',pin='xy_pin'),
                         end_pin=Dict(component='my_connector_xyline01',pin='c_pin_r'),),
                          trace_width='10um',trace_gap='7um',chip = 'C_chip',fillet='10 um')

xyline_q1 = RoutePathfinder(design, 'xyline_q1', options=opts)

opts = Dict(pin_inputs=Dict(start_pin=Dict(component='my_connector_xyline01',pin='c_pin_l'),
                         end_pin=Dict(component='launch_zline03',pin='tie'),),lead = Dict(end_straight = extended_pin_length_xy,
                                                                                     ),
                          trace_width='10um',trace_gap='5um',chip = 'C_chip',fillet='10 um')

xyline_lz03 = RoutePathfinder(design, 'xyline_lz03', options=opts)

gui.rebuild()
gui.autoscale()


#caculate connector position
# c_length= 0.1
# x_1 = y_1-Q_chip_size*math.sqrt(2)/2
#two short ground position


# x,y = r1.make_ro()
# x, y=-x,-y
# x, y

In [23]:
my_xyline01.pins.xy_pin.middle[1],my_xyline01.options.pos_y

(1.255, 1.355)

In [24]:
def  xyline_creator(qubit_id, launch_pad_name, xyline_position=np.array([-2/3,0]),l_1=0.1,l_2=0.15,l_3=0.05,angle=-90, inverse=False,mirror=False,type=1, width_right='10um',gap_right='7um',width_left='10um',gap_left='5um'):
        """qubit_id : integral id number
             xyline_position: numpy array assuming unit length of x y z direction of qubit
             l_1,l_2,l_3: three parts of xyline
             """
        # xyline_x0_1 = qubit_pos_list[qubit_id][0]-1/3*qubit_list[qubit_id].parse_options().cross_width
        # xyline_y0_1 = qubit_pos_list[qubit_id][1]

        xyline_x0_1, xyline_y0_1 = np.array([ qubit_pos_list[qubit_id][0],qubit_pos_list[qubit_id][1]])+xyline_position/2*qubit_list[qubit_id].parse_options().cross_width
        opt = Dict(pos_x=xyline_x0_1,
                               pos_y=xyline_y0_1,
                               l_1=l_1,
                               l_2=l_2,
                               l_3 = l_3,
                               flux_cpw_width='5 um',
                               flux_cpw_gap='3 um',
                               flux_cpw_width0=width_right,
                               flux_cpw_gap0=gap_right,
                               c_length = '15 um',
                               angle = angle,
                               fillet = '1 um',
                               orientation='90',
                               layer='1',
                               layer_subtract='2',
                               inverse = inverse,
                               mirror = mirror,
                               subtract=True,
                               chip='C_Chip',
                             )

        my_xyline01 = MyXYLine01(design,'my_xyline01',options=opt)

        xyline_length =my_xyline01.pins.xy_pin.middle[1]-my_xyline01.options.pos_y
        xyline01_end_pos_y = xyline_y0_1+xyline_length
        xyline01_c_pos_x, xyline01_c_pos_y = get_connector_pos_xy(xyline01_end_pos_y,Q_chip_size,type)
        extended_pin_length_xy= (xyline01_end_pos_y-design.components[launch_pad_name].pins.tie.middle[1])/design.components[launch_pad_name].pins.tie.normal[1]
        opt = Dict(pos_x=xyline01_c_pos_x,
                                   pos_y=xyline01_c_pos_y,
                                   width=width_right,
                                   gap=gap_right,
                                   width0=width_left,
                                   gap0=gap_left,
                                   length = '100 um',
                                   orientation='0',
                                   layer='1',
                                   layer_subtract='2',
                                   subtract=True,
                                   chip='C_Chip', )
        my_connector_xyline01 = MyConnector(design,'my_connector_xyline01',options=opt)

        opts = Dict(pin_inputs=Dict(start_pin=Dict(component='my_xyline01',pin='xy_pin'),
                                 end_pin=Dict(component='my_connector_xyline01',pin='c_pin_r'),),
                                  trace_width=width_right,trace_gap=gap_right,chip = 'C_chip',fillet='10 um')

        xyline_q1 = RoutePathfinder(design, 'xyline_q1', options=opts)

        opts = Dict(pin_inputs=Dict(start_pin=Dict(component='my_connector_xyline01',pin='c_pin_l'),
                                 end_pin=Dict(component=launch_pad_name,pin='tie'),),lead = Dict(end_straight = extended_pin_length_xy,),
                                  trace_width=width_left,trace_gap=gap_left,chip = 'C_chip',fillet='10 um')

        xyline_lz03 = RoutePathfinder(design, 'xyline_lz03', options=opts)
        return my_connector_xyline01, xyline_q1, xyline_lz03

xyline_creator(qubit_id=0, launch_pad_name='launch_zline03')
# opt = Dict(pos_x=x_1,
#                            pos_y=y_1,
#                            width='10 um',
#                            gap='7 um',
#                            width0='10 um',
#                            gap0='5 um',
#                            length = '100 um',
#                            orientation='0',
#                            layer='1',
#                            layer_subtract='2',
#                            subtract=True,
#                            chip='C_Chip', )
# my_connector1 = MyConnector(design,'my_connector1',options=opt)
# gui.rebuild()
# gui.autoscale()
# stg1_ai = ShortToGround(design, 'short1_ai', options=Dict(pos_x=x_12,  pos_y=y_12, orientation='180',))
# stg1_ao = ShortToGround(design, 'short1_ao', options=Dict(pos_x=x_12,  pos_y=y_12, orientation='0',))
# stg1_bi = ShortToGround(design, 'short1_bi', options=Dict(pos_x=x_11,  pos_y=y_11, orientation='180',))
# stg1_bo = ShortToGround(design, 'short1_bo', options=Dict(pos_x=x_11,  pos_y=y_11, orientation='0',))
#
# stg1_ai.options.width = '10 um'
# stg1_ai.options.gap = '7 um'
# stg1_ao.options.width = '10 um'
# stg1_ao.options.gap = '7 um'
# stg1_bi.options.width = '10 um'
# stg1_bi.options.gap = '5 um'
# stg1_bo.options.width = '10 um'
# stg1_bo.options.gap = '5 um'

# stg1_bi.options.width = '10 um'
# stg1_bo.options.gap = '5 um'


# readout_line_ydistance = 0.15
# readout_line_xdistance = 0.5
# pad_pos_y0 = y-readout_line_ydistance
# pad_pos_x0 = x -readout_line_xdistance
# pad_pos_y1 = pad_pos_y0
# pad_pos_x1 = x+4*readout_line_xdistance
# pin_opt = Dict(hfss_wire_bonds=True, pin_inputs=Dict(start_pin=Dict(component='Launch_Readout0',pin='tie'),
#                          end_pin=Dict(component='Launch_Readout1',pin='tie'),),
#                           trace_width='10um',trace_gap='6um')
# pin_opt.fillet = '30 um'
# pin_opt.lead.start_straight = '90 um'
# pin_opt.lead.end_straight = '90 um'
#
# jogsS = OrderedDict()
# jogsS[0] = ["R", '90um']
# jogsE = OrderedDict()
# jogsE[0] = ["L", '90um']
#
# launch_readout0 = LaunchpadWirebond(design,'Launch_Readout0',options=Dict(pos_x=pad_pos_x0,pos_y=pad_pos_y0,orientation='90',chip = 'C_chip'))
# launch_readout1 = LaunchpadWirebond(design,'Launch_Readout1',options=Dict(pos_x=pad_pos_x1,pos_y=pad_pos_y1,orientation='90',chip = 'C_chip'))
# t1_readout_q_01=RoutePathfinder(design,'T1_Readout_Q_01',options=pin_opt)
gui.rebuild()
gui.autoscale()

In [25]:
# my_connector1.pins

In [26]:
# # design.delete_all_components()
# opt = Dict(pos_x=qubit_pos_list[0][0], pos_y=qubit_pos_list[0][1] ,  end_yposition=y1_zline,
#            inverse = True, mirror = True, flux_cpw_gap0 = '7 um', end_horizontal_length = '100 um')
# my_flux01 = MyFluxLine02(design,'my_flux01',options=opt)
# # my_flux.pins.flux_pin.normal =[0,-1]
# gui.rebuild()
# gui.autoscale()


In [27]:

# my_flux01.options
my_flux01.pins
# # stg1_ai.pins.short.gap=0.007
# stg1_ai.pins.short.gap
# # stg1_ai.options
# design.components['short1_ai'].pins
# design.template_options

{'flux_pin': {'points': array([[-3.26471469,  1.82721469],
         [-3.26471469,  1.83721469]]),
  'middle': array([-3.26471469,  1.83221469]),
  'normal': array([-1.,  0.]),
  'tangent': array([0., 1.]),
  'width': 0.01,
  'gap': 0.007,
  'chip': 'main',
  'parent_name': 93,
  'net_id': 91,
  'length': 0}}

In [28]:
pin_opt = Dict(pin_inputs=Dict(start_pin=Dict(component='my_flux01',pin='flux_pin'),
                         end_pin=Dict(component='my_connector1',pin='c_pin_r'),),trace_width = '10 um',
                        trace_gap='7 um',chip = 'C_chip')

In [29]:
path_zline_q1 = RouteStraight(design, 'path_zline_q1', options=pin_opt)
gui.rebuild()
gui.autoscale()

11:18AM 53s WARNING [__init__]: Pin flux_pin of component my_flux01 is already in use. path_zline_q1 has not been built. Please check your pin_input values.


In [30]:
path_zline_q1.options


{'chip': 'C_chip',
 'layer': '1',
 'pin_inputs': {'start_pin': {'component': 'my_flux01', 'pin': 'flux_pin'},
  'end_pin': {'component': 'my_connector1', 'pin': 'c_pin_r'}},
 'fillet': '0',
 'lead': {'start_straight': '0mm',
  'end_straight': '0mm',
  'start_jogged_extension': '',
  'end_jogged_extension': ''},
 'total_length': '7mm',
 'trace_width': '10 um',
 'hfss_wire_bonds': False,
 'q3d_wire_bonds': False,
 'trace_gap': '7 um'}

In [31]:
from collections import OrderedDict
# anchors = OrderedDict()
# anchors[0] = np.array([-0.52,0.52])
# jogsS = OrderedDict()
# jogsS[0] = [45, '10um']
# options = {'pin_inputs':
#             {'start_pin': {'component': 'short2o', 'pin': 'short'},
#              'end_pin': {'component': 'launch_zline', 'pin': 'tie'}},
#             'lead': {'start_straight': '491.5 um', },
#             'step_size': '0.6mm',
#             # 'anchors': anchors,
#             'fillet': '90 um',
#            'chip': 'C_chip'
#            }
opts = Dict(pin_inputs=Dict(start_pin=Dict(component='my_connector1',pin='c_pin_l'),
                         end_pin=Dict(component='launch_zline04',pin='tie'),),lead = Dict(end_straight = extended_pin_length,
                                                                                     ),
                          trace_width='10um',trace_gap='5um',chip = 'C_chip',fillet='10 um')

path_zline_l1 = RoutePathfinder(design, 'path_zline_l1', options=opts)
gui.rebuild()
gui.autoscale()

11:19AM 01s WARNING [__init__]: Component my_connector1 does not exist. path_zline_l1 has not been built. Please check your pin_input values.


In [32]:
xyline_length = 0.1
extended_pin_length_xy= (qubit_pos_list[0][1]-xyline_length-launch_list[3].pins.tie.middle[1])/launch_list[3].pins.tie.normal[1]
# xyline_y_1 = launch_list[3].pins.tie.middle[1]+extended_pin_length*launch_list[3].pins.tie.normal[1]
# xyline_rtq_y_1 = abs(xyline_y_1-qubit_pos_list[0][1])
# xyline_rtq_y_1,launch_list[3].pins.tie.middle[1],extended_pin_length*launch_list[3].pins.tie.normal[1],qubit_pos_list[0][1]
extended_pin_length_xy

0.19201913522726752

In [33]:
xyline_x0_1 = qubit_pos_list[0][0]-1/3*qubit_list[0].parse_options().cross_width
xyline_y0_1 = qubit_pos_list[0][1]
opt = Dict(pos_x=xyline_x0_1,
                       pos_y=xyline_y0_1,
                       l_1='100 um',
                       l_2='150 um',
                       l_3 = '50 um',
                       flux_cpw_width='5 um',
                       flux_cpw_gap='3 um',
                       flux_cpw_width0='10 um',
                       flux_cpw_gap0='7 um',
                       c_length = '15 um',
                       angle = '-90',
                       fillet = '1 um',
                       orientation='90',
                       layer='1',
                       layer_subtract='2',
                       inverse = False,
                       mirror = False,
                       subtract=True,
                       chip='C_Chip',
                     )

my_xyline01 = MyXYLine01(design,'my_xyline01',options=opt)
# # my_flux.pins.flux_pin.normal =[0,-1]
gui.rebuild()
gui.autoscale()
# gui.screenshot()
# design.overwrite_enabled = True

In [34]:
opts = Dict(pin_inputs=Dict(start_pin=Dict(component='my_xyline01',pin='xy_pin'),
                         end_pin=Dict(component='launch_zline03',pin='tie'),),lead = Dict(end_straight = extended_pin_length_xy,
                                                                                     ),
                          trace_width='10um',trace_gap='5um',chip = 'C_chip',fillet='10 um')

path_xyline_l1 = RoutePathfinder(design, 'path_xyline_l1', options=opts)
gui.rebuild()
gui.autoscale()
# qubit_pos_list[0][1]
#draw zline----first part before connector

# def  get_position(pins,length):
#     eps = 1e-2
#     pin = Dict()
#     pin.middle = pins.middle
#     pin.normal = pins.normal
#     z_line = QRouteLead()
#     z_line.seed_from_pin(pin=pin)
#     z_line.go_straight(eps)
#     z_line.go_angle(length,45)
#     return  z_line.get_tip()
# extended_zline_length = 0.19
# connector_length = 0.015
# connector_total_length = extended_zline_length+connector_length
# pos_x = get_position(my_flux01.pins.flux_pin,extended_zline_length).position[0]
# pos_y = get_position(my_flux01.pins.flux_pin,extended_zline_length).position[1]
# pos_x0 = get_position(my_flux01.pins.flux_pin,connector_total_length).position[0]
# pos_y0 = get_position(my_flux01.pins.flux_pin,connector_total_length).position[1]
# pos_x
# my_flux.pins.flux_pin.middle
# pins =my_flux.pins.middle
# pin = Dict()
# # pin.middle = pins.middle
# # pin.normal = pins.normal
# pins




11:19AM 17s WARNING [__init__]: Pin tie of component launch_zline03 is already in use. path_xyline_l1 has not been built. Please check your pin_input values.


In [132]:
res1_readout_y = design.components['R1'].pins.readout.middle[1]


# # set connector pins on both sides
#
# from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
# from  qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
#
# otg11 = ShortToGround(design, 'short1i', options=Dict(pos_x=pos_x,  pos_y=pos_y, orientation='135', chip ='C_chip'),)
# otg12 = ShortToGround(design, 'short1o', options=Dict(pos_x=pos_x,  pos_y=pos_y, orientation='-45', chip ='C_chip'),)
# otg21 = ShortToGround(design, 'short2i', options=Dict(pos_x=pos_x0,  pos_y=pos_y0, orientation='135',chip = 'C_chip'))
# otg22 = ShortToGround(design, 'short2o', options=Dict(pos_x=pos_x0,  pos_y=pos_y0, orientation='-45',chip = 'C_chip'))
# # otg21 = ShortToGround(design, 'short2i', options=Dict(pos_x='15 um',  pos_y='1mm', orientation='180'))
# # otg22 = ShortToGround(design, 'short2o', options=Dict(pos_x='25 um',  pos_y='1mm', orientation='0'))
# # def  get_position(pin, length):
#
# otg11.options.width = '5 um'
# otg11.options.gap = '3 um'
# otg12.options.width = '5 um'
# otg12.options.gap = '3 um'
# gui.rebuild()
# gui.autoscale()
# gui.zoom_on_components(['short1i'])

array([-3.356,  0.56 ])

In [37]:
# otg22.pins

{'short': {'points': array([[-0.16199242,  0.18742136],
         [-0.15492136,  0.19449242]]),
  'middle': array([-0.15845689,  0.19095689]),
  'normal': array([-0.70710678,  0.70710678]),
  'tangent': array([0.70710678, 0.70710678]),
  'width': 0.01,
  'gap': 0.006,
  'chip': 'C_chip',
  'parent_name': 70,
  'net_id': 0,
  'length': 0}}

In [38]:
#draw zline ---first part before connector

# jogsS = OrderedDict()
# jogsS[0] = [45, '6um']
#
# pin_opt = Dict(pin_inputs=Dict(start_pin=Dict(component='my_flux',pin='flux_pin'),
#                          end_pin=Dict(component='short1i',pin='short'),),lead = Dict(start_straight='10 um',end_straight='150 um',
#                                                                                      start_jogged_extension = jogsS),
#                           trace_width='5um',trace_gap='3um',chip = 'C_chip',fillet='10 um')
# # pin_opt = Dict(pin_inputs=Dict(start_pin=Dict(component='short1o',pin='short'),
# #                          end_pin=Dict(component='short1i',pin='short'),),
# #                         chip = 'C_chip')
#
# flux_line=RouteStraight(design,'flux_line',options=pin_opt)
#
# gui.rebuild()
# gui.autoscale()

In [39]:
# flux_line.options

{'chip': 'C_chip',
 'layer': '1',
 'pin_inputs': {'start_pin': {'component': 'my_flux', 'pin': 'flux_pin'},
  'end_pin': {'component': 'short1i', 'pin': 'short'}},
 'fillet': '10 um',
 'lead': {'start_straight': '10 um',
  'end_straight': '150 um',
  'start_jogged_extension': {0: [45, '6um']},
  'end_jogged_extension': ''},
 'total_length': '7mm',
 'trace_width': '5um',
 'hfss_wire_bonds': False,
 'q3d_wire_bonds': False,
 'trace_gap': '3um',
 '_actual_length': '0.19570796266628024 mm'}

In [40]:
#draw connector
#
# from qiskit_metal.qlibrary.user_components.my_qcomponent import RouteConnector
# pin_opt = Dict(pin_inputs=Dict(start_pin=Dict(
#                                     component='short1o',
#                                     pin='short'),
#                                 end_pin=Dict(
#                                     component='short2i',
#                                     pin='short')),chip = 'C_chip')
# my_connector = RouteConnector(design,'my_connector',pin_opt)
#
# # pin_opt = Dict(pin_inputs=Dict(start_pin=Dict(
# #                                     component='open2i',
# #                                     pin='short'),
# #                                 end_pin=Dict(
# #                                     component='open2o',
# #                                     pin='short')),chip = 'C_chip')
# # my_connector1 = RouteConnector(design,'my_connector1',pin_opt)
# gui.rebuild()
# gui.autoscale()

In [41]:
# def  get_position(pins,length):
#     eps = 1e-2
#     pin = Dict()
#     pin.middle = pins.middle
#     pin.normal = pins.normal
#     z_line = QRouteLead()
#     z_line.seed_from_pin(pin=pin)
#     z_line.go_straight(length)
#     z_line.go_angle(eps,45)
#     return  z_line.get_tip()
# extended_zline_length0 = 0.2
# pos_x_zline = get_position(otg22.pins.short,extended_zline_length0).position[0]
# pos_y_zline = get_position(otg22.pins.short,extended_zline_length0).position[1]
# pos_y_zline

0.3323782462

In [42]:
opt=Dict(pos_x=-3 , pos_y=pos_y_zline, orientation='-45', pad_width='245 um', pad_height='245 um', pad_gap = '100 um', lead_length = '176 um', chip = 'C_chip')
# test = OpenToGround(design, 'open01', options=Dict(pos_x='-3 mm',  pos_y=pos_y_zline+0.02, orientation='-45', chip ='C_chip'),)
launch_zline = LaunchpadWirebond(design,'launch_zline',options=opt)
end_jog_length = 0.02
launch_zline.options.pos_y = pos_y_zline+launch_zline.parse_options().lead_length*math.cos(launch_zline.parse_options().orientation/180*math.pi)+end_jog_length*math.cos(launch_zline.parse_options().orientation/180*math.pi)


gui.rebuild()
gui.autoscale()

In [43]:
launch_zline.options

{'pos_x': -3,
 'pos_y': 0.47097117531256333,
 'orientation': '-45',
 'chip': 'C_chip',
 'layer': '1',
 'trace_width': 'cpw_width',
 'trace_gap': 'cpw_gap',
 'lead_length': '176 um',
 'pad_width': '245 um',
 'pad_height': '245 um',
 'pad_gap': '100 um',
 'taper_height': '122um'}

In [44]:
launch_zline.pins

{'tie': {'points': array([[-2.87201367,  0.35005592],
         [-2.87908474,  0.34298485]]),
  'middle': array([-2.87554921,  0.34652038]),
  'normal': array([ 0.70710683, -0.70710673]),
  'tangent': array([-0.70710673, -0.70710683]),
  'width': 0.01,
  'gap': 0.006,
  'chip': 'C_chip',
  'parent_name': 73,
  'net_id': 0,
  'length': 0}}

In [45]:
from collections import OrderedDict
# anchors = OrderedDict()
# anchors[0] = np.array([-0.52,0.52])
jogsS = OrderedDict()
jogsS[0] = [45, '10um']
# options = {'pin_inputs':
#             {'start_pin': {'component': 'short2o', 'pin': 'short'},
#              'end_pin': {'component': 'launch_zline', 'pin': 'tie'}},
#             'lead': {'start_straight': '491.5 um', },
#             'step_size': '0.6mm',
#             # 'anchors': anchors,
#             'fillet': '90 um',
#            'chip': 'C_chip'
#            }
opts = Dict(pin_inputs=Dict(start_pin=Dict(component='short2o',pin='short'),
                         end_pin=Dict(component='launch_zline',pin='tie'),),lead = Dict(start_straight=extended_zline_length0, end_straight = end_jog_length,
                                                                                     start_jogged_extension = jogsS),
                          trace_width='10um',trace_gap='6um',chip = 'C_chip',fillet='10 um')

qa = RoutePathfinder(design, 'path_zline', options=opts)
gui.rebuild()
gui.autoscale()


In [20]:
a_gds = design.renderers.gds
a_gds.options
# from  qiskit_metal.qlibrary.user_components.my_qcomponent import  New_Transomon_Cross
# my_test = New_Transomon_Cross(design,'test_chip',options=Dict(gds_cell_name='FakeJunction_01'))
# # my_test1 = MyQComponent(design,'test_chip1')
# gui.rebuild()
# gui.autoscale()
# gui.screenshot()
# design.overwrite_enabled = True

{'short_segments_to_not_fillet': 'True',
 'check_short_segments_by_scaling_fillet': '2.0',
 'gds_unit': 0.001,
 'ground_plane': 'True',
 'negative_mask': {'main': []},
 'fabricate': 'False',
 'corners': 'circular bend',
 'tolerance': '0.00001',
 'precision': '0.000000001',
 'width_LineString': '10um',
 'path_filename': '../resources/Fake_Junctions.GDS',
 'junction_pad_overlap': '5um',
 'max_points': '199',
 'cheese': {'datatype': '100',
  'shape': '0',
  'cheese_0_x': '25um',
  'cheese_0_y': '25um',
  'cheese_1_radius': '100um',
  'view_in_file': {'main': {1: True}},
  'delta_x': '100um',
  'delta_y': '100um',
  'edge_nocheese': '200um'},
 'no_cheese': {'datatype': '99',
  'buffer': '25um',
  'cap_style': '2',
  'join_style': '2',
  'view_in_file': {'main': {1: True}}},
 'bounding_box_scale_x': '1.2',
 'bounding_box_scale_y': '1.2'}

In [24]:
a_gds.options['short_segments_to_not_fillet'] = 'True'
scale_fillet = 2.0
a_gds.options['check_short_segments_by_scaling_fillet'] = scale_fillet
a_gds.options.negative_mask = {'Q_chip': []}
a_gds.options.cheese.view_in_file =  {'Q_chip': {1: False}, 'C_chip':{1:False, 2:False}}
a_gds.options.no_cheese.view_in_file =  {'Q_chip': {1: False}, 'C_chip':{1:False, 2: False}}
a_gds.export_to_gds('flip_chip_test03.gds')

03:39PM 49s WARNING [get_x_y_for_chip]: Chip name "main" is not in self._chips dict. Return "None" in tuple.
03:39PM 49s WARNING [_create_qgeometry_for_gds]: design.get_x_y_for_chip() did NOT return a good code for chip=main,for ground subtraction-box using the size calculated from QGeometry, ((-1.465765498, -0.92, 1.034234502, 0)) will be used. 
03:39PM 49s WARNING [_import_junctions_to_one_cell]: Not able to find file:"../resources/Fake_Junctions.GDS".  Not used to replace junction. Checked directory:"C:\Users\CJZ\PycharmProjects\resources".
03:39PM 49s WARNING [_import_junctions_to_one_cell]: Not able to find file:"../resources/Fake_Junctions.GDS".  Not used to replace junction. Checked directory:"C:\Users\CJZ\PycharmProjects\resources".
03:39PM 49s WARNING [_import_junctions_to_one_cell]: Not able to find file:"../resources/Fake_Junctions.GDS".  Not used to replace junction. Checked directory:"C:\Users\CJZ\PycharmProjects\resources".
03:39PM 49s WARNING [_check_either_cheese]: Chip

1

In [94]:
design.chips


{'C_chip': {'material': 'silicon',
  'layer_start': '0',
  'layer_end': '2048',
  'size': {'center_x': '0.0mm',
   'center_y': '0.0mm',
   'center_z': '0.0mm',
   'size_x': '9mm',
   'size_y': '9mm',
   'size_z': '-280um',
   'sample_holder_top': '890um',
   'sample_holder_bottom': '1650um'}},
 'Q_chip': {'material': 'silicon',
  'layer_start': '0',
  'layer_end': '2048',
  'size': {'center_x': '0.0mm',
   'center_y': '0.0mm',
   'center_z': '20 um',
   'size_x': '9mm',
   'size_y': '9mm',
   'size_z': '280um',
   'sample_holder_top': '890um',
   'sample_holder_bottom': '1650um'}}}

In [99]:
design.qgeometry.get_element_types()

['path', 'poly', 'junction']

In [110]:
design.qgeometry.tables['poly']

,component,name,geometry,layer,subtract,helper,chip,fillet
0,1,cross_0,"POLYGON ((0.01646 -0.00939, 0.03284 0.00698, 0...",1,False,False,Q_chip,NaN
1,1,cross_1,"POLYGON ((-0.61309 0.01962, -0.61264 0.01941, ...",1,False,False,Q_chip,NaN
2,1,cross_etch,"POLYGON ((-0.66000 -0.04000, -0.66098 -0.03995...",1,True,False,Q_chip,NaN
3,2,ro,"POLYGON ((0.28524 0.00490, 0.28596 0.00975, 0....",1,False,False,C_chip,NaN
4,2,ro_etch,"POLYGON ((0.34750 -0.09012, 0.34748 -0.09132, ...",2,True,False,C_chip,NaN
5,3,launch_pad,"POLYGON ((-1.47077 -0.94500, -1.50577 -1.06700...",1,False,False,C_chip,NaN
6,3,pocket,"POLYGON ((-1.47677 -0.94500, -1.56377 -1.06700...",1,True,False,C_chip,NaN
7,4,launch_pad,"POLYGON ((1.02923 -0.94500, 0.99423 -1.06700, ...",1,False,False,C_chip,NaN
8,4,pocket,"POLYGON ((1.02323 -0.94500, 0.93623 -1.06700, ...",1,True,False,C_chip,NaN


In [111]:
design.chip_info

AttributeError: 'DesignFlipChip' object has no attribute 'chip_info'